#RECESSION ANALYSIS

###Rcession is an economic situation where circulation of money is in the economy is low for 2 consecutive quarters.

In [2]:
#load file from the system
from google.colab import files
uploaded = files.upload()

Saving UK_monthly_gdp.csv to UK_monthly_gdp (1).csv


In [3]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.templates='plotly_white'

#read the csv file
data=pd.read_csv('/content/UK_monthly_gdp.csv')
print(data.head())

  Time Period  GDP Growth
0    /01/2020         0.3
1    /02/2020        -0.5
2    /03/2020        -7.0
3    /04/2020       -20.9
4    /05/2020         3.2


In [7]:
#growth of GDP over time using heatmap
fig = go.Figure(data=go.Heatmap(
                   z=[data['GDP Growth']],
                   x=data.index,
                   y=['GDP Growth'],
                   colorscale='Viridis'))
fig.update_layout(title='GDP Growth Over Time', xaxis_title='Time Period', yaxis_title='')
fig.show()

In [8]:
#recession is calculated depending on the circulation of money in two consecutive quarters.
#given data has to be converted from monthly to quarterly format
#we use resampling for the same

data['Time Period']= pd.to_datetime(data['Time Period'], format='/%m/%Y')
data.set_index('Time Period', inplace=True)

#quarterly data
quarterly_data=data.resample('Q').mean()
print(quarterly_data.head())

             GDP Growth
Time Period            
2020-03-31    -2.400000
2020-06-30    -2.900000
2020-09-30     3.500000
2020-12-31     0.200000
2021-03-31     0.033333


In [10]:
#now the recession can be calculated using the quaterly data

quarterly_data['Recession']= ((quarterly_data['GDP Growth']<0) & (quarterly_data['GDP Growth'].shift(1)<0))

#filling the missing values with False(because the 1st quarter cant be in a recession)
quarterly_data['Recession'].fillna(False, inplace=True)

#plot the growth and recession data
fig=go.Figure()
fig.add_trace(go.Scatter(x=quarterly_data.index,
                         y=quarterly_data['GDP Growth'],
                         name='GDP Growth',
                         line=dict(color='green', width=3)))

fig.add_trace(go.Scatter(x=quarterly_data[quarterly_data['Recession']].index,
                         y=quarterly_data[quarterly_data['Recession']]['GDP Growth'],
                         name='Recession',
                         line=dict(color='red', width=3)))

fig.update_layout(title='GDP Growth and Recession Over Time(Quarterly)',
                  xaxis_title='Time Period',
                  yaxis_title='GDP Growth')

fig.show()

###1. The red line shows the periods of negative GDP Growth(considered recessions)

###2. The green line shows the overall trend in the GDP growth over time

###Severity of Recession is the extent to which the economy contracts during a recession. A severe recession onvolves a deeper and more prolonged decline in economic activity, resulting in negative effects on employment, incomes, etc. 

In [11]:
#analyse the severity of recession
quarterly_data['Recession Start']= quarterly_data['Recession'].ne(quarterly_data['Recession'].shift()).cumsum()

recession_period=quarterly_data.groupby('Recession Start')
recession_duration= recession_period.size()
recession_severity=recession_period['GDP Growth'].sum()

fig=go.Figure()
fig.add_trace(go.Bar(x=recession_duration.index,
                     y=recession_duration,
                     name='Recession Duration'))

fig.add_trace(go.Bar(x=recession_severity.index,
                     y=recession_severity,
                     name='Recession Severity'))

fig.update_layout(title='Duration and Severity of Recession',
                  xaxis_title='Recession Periods',
                  yaxis_title='Duration/Severity')